In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import sympy as sp
import handcalcs.render
import forallpeople as si
import matplotlib.pyplot as plt
%matplotlib inline
from math import tan, sin,cos, pi, radians, sqrt, trunc
from IPython.display import display, Markdown
si.environment('structural', top_level=True)

<h2>SL-Deck verification</h2>

<h3>Compartment beneath: Office workplace</h3>
<h4>Loads</h4>

In [2]:
%%render 2

psi_11 = 0.4
G_k = (3.88+1.21)*kN/m**2
Q_k = 4.81*kN/m**2

<IPython.core.display.Latex object>

Load combination for fire:

In [3]:
%%render 2
q_d = G_k+psi_11*Q_k

<IPython.core.display.Latex object>

<h4>Slab parameters</h4>

In [4]:
%%render param 2

L = 8.4*m
d = 270*mm
w = 2400*mm
fck_20 = 55*MPa
fp_20 = 1634*MPa
phi_s = 12.5*mm
A_w = 93*mm**2
A_p = 14*A_w
d_s = d - 60*mm
E_s20 = 210*GPa


<IPython.core.display.Latex object>

From CONFIRE

<img src = "Confire3.png" width=400>

In [5]:
%%render param 2

T_max = (207*dC)
T_cold = (207*dC) 
xi_CMH = 1
xi_CMC = 0.9914

<IPython.core.display.Latex object>

In [6]:
cte = pd.read_excel('strength-reduction-constants.xlsx')

In [7]:
def material_constants(mat_index):
    k = cte['k '][mat_index]
    T1 = cte['T1 '][mat_index]*dC
    T2 = cte['T2 '][mat_index]*dC
    T8 = cte['T8 '][mat_index]*dC
    T64 = cte['T64'][mat_index]*dC
    return k, T1, T2, T8, T64



In [8]:
from handcalcs import handcalc
@handcalc(jupyter_display = True)
def material_deterioration(T,k,T1,T2,T8,T64):
    """
    Calculate the deterioration of a material.
    """
    xi = k + (1-k)/(1+T/T1+(T/T2)**2+(T/T8)**8+(T/T64)**64)
    return xi


In [9]:
k, T1, T2, T8, T64 = material_constants(8)
xi_sH = material_deterioration(T_max,k,T1,T2,T8,T64)


<IPython.core.display.Latex object>

In [10]:
%%render 2
xi_sH

<IPython.core.display.Latex object>

In [11]:
k, T1, T2, T8, T64 = material_constants(10)
xi_sC = material_deterioration(T_cold,k,T1,T2,T8,T64)

<IPython.core.display.Latex object>

In [12]:
%%render 2
xi_sC

<IPython.core.display.Latex object>

**Before fire**

In [13]:
%%render 2

M_max_ALS = q_d*L**2/8*w
F_su20 = A_p*fp_20
y_20 = F_su20/(fck_20*w)
M_u20 = F_su20*(d_s-y_20/2)
mu_20 = M_max_ALS/M_u20

<IPython.core.display.Latex object>

**HOT - During fire**

In [18]:
%%tex symbolic

F_suH = xi_sH*F_su20
y_H = F_suH/(xi_CMH*fck_20*w)
M_u_H = F_suH*(d_s-0.5*y_H)
mu_H = M_max_ALS/M_u_H

\[
\begin{aligned}
F_{suH} &= \xi_{sH} \cdot F_{su20} \; 
\\[10pt]
y_{H} &= \frac{ F_{suH} }{ \xi_{CMH} \cdot \mathrm{fck}_{20} \cdot w } \; 
\\[10pt]
M_{u_{H}} &= F_{suH} \cdot \left( d_{s} - 0.5 \cdot y_{H} \right) \; 
\\[10pt]
\mu_{H} &= \frac{ M_{max_{ALS}} }{ M_{u_{H}} } \; 
\end{aligned}
\]


**COLD - After fire**

In [15]:
%%render 2

F_suC = xi_sC*F_su20
y_C = F_suC/(xi_CMC*fck_20*w)
M_u_C = F_suC*(d_s-y_C/2)
mu_C = M_max_ALS/M_u_C

<IPython.core.display.Latex object>

Over-reinforcment check

In [16]:
%%render 2

varepsilon_s = (d_s-5/4*y_C)/(5/4*y_C)*0.35/100/xi_CMC
varepsilon_smin = fp_20/E_s20 + 0.2/100

<IPython.core.display.Latex object>

In [17]:
if varepsilon_s>varepsilon_smin:
    display(Markdown('The slab is verified for fire'))
else:
    display(Markdown('The slab is not verified for fire'))

The slab is verified for fire